<h1>Using R with Amazon SageMaker</h1>

This sample Notebook describes how to train, deploy, and retrieve predictions from a machine learning (ML) model using [Amazon SageMaker](https://aws.amazon.com/sagemaker/) and [R](https://www.r-project.org/). The model predicts abalone age as measured by the number of rings in the shell. The [reticulate](https://rstudio.github.io/reticulate/) package will be used as an R interface to [Amazon SageMaker Python SDK](https://sagemaker.readthedocs.io/en/latest/index.html) to make API calls to Amazon SageMaker. The `reticulate` package translates between R and Python objects, and Amazon SageMaker provides a serverless data science environment to train and deploy ML models at scale.


<h3>Reticulating the Amazon SageMaker Python SDK</h3>

First, load the `reticulate` library and import the `sagemaker` Python module. Once the module is loaded, use the `$` notation in R instead of the `.` notation in Python to use available classes. 

In [1]:
# Turn warnings off globally
options(warn=-1)

In [2]:
# Install Dependencies----
if(!("remotes" %in% rownames(installed.packages()))) install.packages("remotes"); print("remotes")
if(!("arules" %in% rownames(installed.packages()))) install.packages("arules"); print("arules")
if(!("bit64" %in% rownames(installed.packages()))) install.packages("bit64"); print("bit64")
if(!("caTools" %in% rownames(installed.packages()))) install.packages("caTools"); print("caTools")
if(!("combinat" %in% rownames(installed.packages()))) install.packages("combinat"); print("combinat")
if(!("data.table" %in% rownames(installed.packages()))) install.packages("data.table"); print("data.table")
if(!("doParallel" %in% rownames(installed.packages()))) install.packages("doParallel"); print("doParallel")
if(!("e1071" %in% rownames(installed.packages()))) install.packages("e1071"); print("e1071")
if(!("fBasics" %in% rownames(installed.packages()))) install.packages("fBasics"); print("fBasics")
if(!("foreach" %in% rownames(installed.packages()))) install.packages("foreach"); print("foreach")
if(!("forecast" %in% rownames(installed.packages()))) install.packages("forecast"); print("forecast")
if(!("fpp" %in% rownames(installed.packages()))) install.packages("fpp"); print("fpp")
if(!("ggplot2" %in% rownames(installed.packages()))) install.packages("ggplot2"); print("ggplot2")
if(!("gridExtra" %in% rownames(installed.packages()))) install.packages("gridExtra"); print("gridExtra")
if(!("here" %in% rownames(installed.packages()))) install.packages("here"); print("here")
if(!("itertools" %in% rownames(installed.packages()))) install.packages("itertools"); print("itertools")
if(!("lime" %in% rownames(installed.packages()))) install.packages("lime"); print("lime")
if(!("lubridate" %in% rownames(installed.packages()))) install.packages("lubridate"); print("lubridate")
if(!("Matrix" %in% rownames(installed.packages()))) install.packages("Matrix"); print("Matrix")
if(!("MLmetrics" %in% rownames(installed.packages()))) install.packages("MLmetrics"); print("MLmetrics")
if(!("monreg" %in% rownames(installed.packages()))) install.packages("monreg"); print("monreg")
if(!("nortest" %in% rownames(installed.packages()))) install.packages("nortest"); print("nortest")
if(!("RColorBrewer" %in% rownames(installed.packages()))) install.packages("RColorBrewer"); print("RColorBrewer")
if(!("recommenderlab" %in% rownames(installed.packages()))) install.packages("recommenderlab"); print("recommenderlab")
if(!("ROCR" %in% rownames(installed.packages()))) install.packages("ROCR"); print("ROCR")
if(!("pROC" %in% rownames(installed.packages()))) install.packages("pROC"); print("pROC")
if(!("Rcpp" %in% rownames(installed.packages()))) install.packages("Rcpp"); print("Rcpp")
if(!("scatterplot3d" %in% rownames(installed.packages()))) install.packages("scatterplot3d"); print("scatterplot3d")
if(!("stringr" %in% rownames(installed.packages()))) install.packages("stringr"); print("stringr")
if(!("sde" %in% rownames(installed.packages()))) install.packages("sde"); print("sde")
if(!("timeDate" %in% rownames(installed.packages()))) install.packages("timeDate"); print("timeDate")
if(!("tsoutliers" %in% rownames(installed.packages()))) install.packages("tsoutliers"); print("tsoutliers")
if(!("wordcloud" %in% rownames(installed.packages()))) install.packages("wordcloud"); print("wordcloud")
if(!("xgboost" %in% rownames(installed.packages()))) install.packages("xgboost"); print("xgboost")
for (pkg in c("RCurl","jsonlite")) if (! (pkg %in% rownames(installed.packages()))) { install.packages(pkg) }
install.packages("h2o", type = "source", repos = (c("http://h2o-release.s3.amazonaws.com/h2o/latest_stable_R")))


[1] "remotes"
[1] "arules"
[1] "bit64"
[1] "caTools"
[1] "combinat"
[1] "data.table"
[1] "doParallel"
[1] "e1071"
[1] "fBasics"
[1] "foreach"
[1] "forecast"
[1] "fpp"
[1] "ggplot2"
[1] "gridExtra"
[1] "here"
[1] "itertools"
[1] "lime"
[1] "lubridate"
[1] "Matrix"
[1] "MLmetrics"
[1] "monreg"
[1] "nortest"
[1] "RColorBrewer"
[1] "recommenderlab"
[1] "ROCR"
[1] "pROC"
[1] "Rcpp"
[1] "scatterplot3d"
[1] "stringr"
[1] "sde"
[1] "timeDate"
[1] "tsoutliers"
[1] "wordcloud"
[1] "xgboost"


Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [ ]:
remotes::install_github('catboost/catboost', subdir = 'catboost/R-package')
remotes::install_github('AdrianAntico/RemixAutoML', upgrade = FALSE, dependencies = FALSE, force = TRUE)


In [ ]:
# Verify the currently installed packages
installed.packages()

In [ ]:
# Install TicToc to measure code running time
install.packages('tictoc', repos='http://cran.us.r-project.org')
library(tictoc)

In [ ]:
install.packages('graphics', repos='http://cran.us.r-project.org')
library(graphics)

# Added additional R libs (NOT used by this notebook) -pmh
install.packages('ggplot2', repos='http://cran.us.r-project.org')
library(ggplot2)

install.packages('datasets', repos='http://cran.us.r-project.org')
library(datasets)

In [ ]:
library(reticulate)
sagemaker <- import('sagemaker')

<h3>Creating and accessing the data storage</h3>

The `Session` class provides operations for working with the following [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) resources with Amazon SageMaker:

* [S3](https://boto3.readthedocs.io/en/latest/reference/services/s3.html)
* [SageMaker](https://boto3.readthedocs.io/en/latest/reference/services/sagemaker.html)
* [SageMakerRuntime](https://boto3.readthedocs.io/en/latest/reference/services/sagemaker-runtime.html)

Let's create an [Amazon Simple Storage Service](https://aws.amazon.com/s3/) bucket for your data. 

In [ ]:
session <- sagemaker$Session()
bucket <- session$default_bucket()

print (paste("Default bucket name:", bucket))

**Note** - The `default_bucket` function creates a unique Amazon S3 bucket with the following name: 

`sagemaker-<aws-region-name>-<aws account number>`

Specify the IAM role's [ARN](https://docs.aws.amazon.com/general/latest/gr/aws-arns-and-namespaces.html) to allow Amazon SageMaker to access the Amazon S3 bucket. You can use the same IAM role used to create this Notebook:

In [ ]:
role_arn <- sagemaker$get_execution_role()

print (paste("Role ARN:",role_arn))

<h3>Downloading and processing the dataset</h3>

The model uses the [abalone dataset](https://archive.ics.uci.edu/ml/datasets/abalone) from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). First, download the data and start the [exploratory data analysis](https://en.wikipedia.org/wiki/Exploratory_data_analysis). Use tidyverse packages to read the data, plot the data, and transform the data into ML format for Amazon SageMaker:

In [ ]:
library(readr)
data_file <- 'http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data'
abalone <- read_csv(file = data_file, col_names = FALSE)
names(abalone) <- c('sex', 'length', 'diameter', 'height', 'whole_weight', 'shucked_weight', 'viscera_weight', 'shell_weight', 'rings')
head(abalone)

The output above shows that `sex` is a factor data type but is currently a character data type (F is Female, M is male, and I is infant). Change `sex` to a factor and view the statistical summary of the dataset:

In [ ]:
abalone$sex <- as.factor(abalone$sex)
summary(abalone)

The summary above shows that the minimum value for `height` is 0.

Visually explore which abalones have height equal to 0 by plotting the relationship between `rings` and `height` for each value of `sex`:

In [ ]:
library(ggplot2)
options(repr.plot.width = 5, repr.plot.height = 4) 
ggplot(abalone, aes(x = height, y = rings, color = sex)) + geom_point() + geom_jitter()

Alternatively, you can use [RBokeh](https://hafen.github.io/rbokeh/) pachage to create interactive illustrations. According the documenation page:
> "*Bokeh is a visualization library that provides a flexible and powerful declarative framework for creating web-based plots. Bokeh renders plots using HTML canvas and provides many mechanisms for interactivity. Bokeh has interfaces in Python, Scala, Julia, and now R.*"

`rbokeh` comes standard with SageMaker's R kernel. You can import the library as follows:

>`library(rbokeh)`

Below is an exmaple of an interactive version of the above chart using RBokeh.

In [ ]:
# Import rbokeh
library(rbokeh)

In [ ]:
p <- figure() %>%
  ly_points(height, rings, data = abalone,
    hover = list(height, rings))
p

The plot shows multiple outliers: two infant abalones with a height of 0 and a few female and male abalones with greater heights than the rest. Let's filter out the two infant abalones with a height of 0.

In [ ]:
library(dplyr)
abalone <- abalone %>%
  filter(height != 0)

<h3>Preparing the dataset for model training</h3>

The model needs three datasets: one each for training, testing, and validation. First, convert `sex` into a [dummy variable](https://en.wikipedia.org/wiki/Dummy_variable_(statistics)) and move the target, `rings`, to the first column. Amazon SageMaker algorithm require the target to be in the first column of the dataset.

In [ ]:
abalone <- abalone %>%
  mutate(female = as.integer(ifelse(sex == 'F', 1, 0)),
         male = as.integer(ifelse(sex == 'M', 1, 0)),
         infant = as.integer(ifelse(sex == 'I', 1, 0))) %>%
  select(-sex)
abalone <- abalone %>%
  select(rings:infant, length:shell_weight)
head(abalone)

Next, sample 70% of the data for training the ML algorithm. Split the remaining 30% into two halves, one for testing and one for validation:

In [ ]:
abalone_train <- abalone %>%
  sample_frac(size = 0.7)
abalone <- anti_join(abalone, abalone_train)
abalone_test <- abalone %>%
  sample_frac(size = 0.5)
abalone_valid <- anti_join(abalone, abalone_test)

Upload the training and validation data to Amazon S3 so that you can train the model. First, write the training and validation datasets to the local filesystem in .csv format:

In [ ]:
write_csv(abalone_train, 'abalone_train.csv', col_names = FALSE)
write_csv(abalone_valid, 'abalone_valid.csv', col_names = FALSE)

Second, upload the two datasets to the Amazon S3 bucket into the `data` key:

In [ ]:
s3_train <- session$upload_data(path = 'abalone_train.csv', 
                                bucket = bucket, 
                                key_prefix = 'data')
s3_valid <- session$upload_data(path = 'abalone_valid.csv', 
                                bucket = bucket, 
                                key_prefix = 'data')

Finally, define the Amazon S3 input types for the Amazon SageMaker algorithm:

In [ ]:
s3_train_input <- sagemaker$s3_input(s3_data = s3_train,
                                     content_type = 'csv')
s3_valid_input <- sagemaker$s3_input(s3_data = s3_valid,
                                     content_type = 'csv')

<h3>Training the model</h3>

Amazon SageMaker algorithm are available via a [Docker](https://www.docker.com/) container. To train an [XGBoost](https://en.wikipedia.org/wiki/Xgboost) model, specify the training containers in [Amazon Elastic Container Registry](https://aws.amazon.com/ecr/) (Amazon ECR) for the AWS Region.

In [ ]:
# Test access to registry for DeepAR Forecasting (NOT used in this notebook) - pmh
registry <- sagemaker$amazon$amazon_estimator$registry(session$boto_region_name, algorithm='forecasting-deepar')
registry

In [ ]:
registry <- sagemaker$amazon$amazon_estimator$registry(session$boto_region_name, algorithm='xgboost')
registry

XGBOOST_IMAGE = sagemaker$amazon$amazon_estimator$get_image_uri(session$boto_region_name, 'xgboost', repo_version='1.0-1')
XGBOOST_IMAGE

In [ ]:
#registry <- sagemaker$amazon$amazon_estimator$registry(session$boto_region_name, algorithm='xgboost')
XGBOOST_IMAGE_WITH_TAG = sagemaker$amazon$amazon_estimator$get_image_uri(session$boto_region_name, 'xgboost', repo_version='1.0-1-cpu-py3')

XGBOOST_IMAGE_WITH_TAG

In [ ]:
# Use version ':1' to get stable version, use '$latest' for experimental
# Testing version ':1.0-1' for multi-model version - failed
# Testing version '1.0-1-cpu-py3' 
# container <- paste(registry, 'xgboost:1.0-1-cpu-py3', sep='/')

#container <- paste(registry, 'sagemaker-xgboost:1.0-1-cpu-py3', sep='/')

container = '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3'
container

Define an Amazon SageMaker [Estimator](http://sagemaker.readthedocs.io/en/latest/estimators.html), which can train any supplied algorithm that has been containerized with Docker. When creating the Estimator, use the following arguments:
* **image_name** - The container image to use for training
* **role** - The Amazon SageMaker service role
* **train_instance_count** - The number of Amazon EC2 instances to use for training
* **train_instance_type** - The type of Amazon EC2 instance to use for training
* **train_volume_size** - The size in GB of the [Amazon Elastic Block Store](https://aws.amazon.com/ebs/) (Amazon EBS) volume to use for storing input data during training
* **train_max_run** - The timeout in seconds for training
* **input_mode** - The input mode that the algorithm supports
* **output_path** - The Amazon S3 location for saving the training results (model artifacts and output files)
* **output_kms_key** - The [AWS Key Management Service](https://aws.amazon.com/kms/) (AWS KMS) key for encrypting the training output
* **base_job_name** - The prefix for the name of the training job
* **sagemaker_session** - The Session object that manages interactions with Amazon SageMaker API

In [ ]:
s3_output <- paste0('s3://', bucket, '/output')
estimator <- sagemaker$estimator$Estimator(image_name = container,
                                           role = role_arn,
                                           train_instance_count = 1L,
                                           train_instance_type = 'ml.m5.large',
                                           train_volume_size = 30L,
                                           train_max_run = 3600L,
                                           input_mode = 'File',
                                           output_path = s3_output,
                                           output_kms_key = NULL,
                                           base_job_name = NULL,
                                           sagemaker_session = NULL,
                                           enable_network_isolation = FALSE) # added

**Note** - The equivalent to `None` in Python is `NULL` in R.

Specify the [XGBoost hyperparameters](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html) and fit the model. Set the number of rounds for training to 100 which is the default value when using the XGBoost library outside of Amazon SageMaker. Also specify the input data and a job name based on the current time stamp:

### Note: Following Cell can take up to 5 minutes...

In [ ]:
tic("Model Fitting")

estimator$set_hyperparameters(num_round = 100L)
job_name <- paste('sagemaker-train-xgboost', format(Sys.time(), '%H-%M-%S'), sep = '-')
input_data <- list('train' = s3_train_input,
                   'validation' = s3_valid_input)

estimator$fit(inputs = input_data,
              job_name = job_name)
toc()

Once training has finished, Amazon SageMaker copies the model binary (a gzip tarball) to the specified Amazon S3 output location. Get the full Amazon S3 path with this command:

In [ ]:
estimator$model_data
estimator

<h3>Deploying the model</h3>

Amazon SageMaker lets you [deploy your model](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html) by providing an endpoint that consumers can invoke by a secure and simple API call using an HTTPS request. Let's deploy our trained model to a `ml.t2.medium` instance.

### Note: Following Cell can take up to 5 minutes...

In [ ]:
#tic("Model Deployment")
#model_endpoint <- estimator$deploy(initial_instance_count = 1L,
#                                   instance_type = 'ml.t2.medium')
#toc()


<h3>Generating predictions with the model</h3>

Use the test data to generate predictions. Pass comma-separated text to be serialized into JSON format by specifying `text/csv` and `csv_serializer` for the endpoint:

In [ ]:
#model_endpoint$content_type <- 'text/csv'
#model_endpoint$serializer <- sagemaker$predictor$csv_serializer
#model_endpoint

Remove the target column and convert the first 500 observations to a matrix with no column names:

In [ ]:
abalone_test <- abalone_test[-1]
num_predict_rows <- 500
test_sample <- as.matrix(abalone_test[1:num_predict_rows, ])
dimnames(test_sample)[[2]] <- NULL


**Note** - 500 observations was chosen because it doesn't exceed the endpoint limitation.

Generate predictions from the endpoint and convert the returned comma-separated string:

In [ ]:
# Import 'stringr'
library(stringr)

In [ ]:
# Invoke the model endpoint via predict
tic("Invoke Endpoint")
predictions <- model_endpoint$predict(test_sample)
predictions <- str_split(predictions, pattern = ',', simplify = TRUE)
predictions <- as.numeric(predictions)
toc()

Column-bind the predicted rings to the test data:

In [ ]:
# Convert predictions to Integer
abalone_test <- cbind(predicted_rings = as.integer(predictions), 
                      abalone_test[1:num_predict_rows, ])
head(abalone_test)

### Additional steps - creation of multi-variant endpoints

The following steps demonstrate how to leverage more deployment options, including how to create Multi-Variant Endpoints, crearing the EndpointConfig structure that enables it, describing the endpoint, etc.

All cells added by Paul Hargis are annotated with "-pmh" in the comment line

In [ ]:
# Install paws package -pmh
install.packages('paws')
library(paws)

In [ ]:
# Create a paws SageMaker session -pmh
sm_client <- paws::sagemaker()

In [ ]:
# Access info from the training job -pmh
# Note: Cut/paste new training job name from SageMaker Dashboard

# new_training_job_name = '<INSERT TRAINING JOB>'
new_training_job_name = 'sagemaker-train-xgboost-22-27-51'
info <- sm_client$describe_training_job(TrainingJobName=new_training_job_name)

In [ ]:
# Access the model artifacts -pmh
model_artifacts <- info[['ModelArtifacts']]
model_data <- model_artifacts[['S3ModelArtifacts']]
print(model_data)

In [ ]:
container

In [ ]:
multi_model_from_estimator = estimator$create_model(role=role_arn, image=container)

In [ ]:
# print model
print(multi_model_from_estimator)

In [ ]:
# Identify new endpoint name -pmh
ENDPOINT_CONFIG_NAME = 'MultiModelEndPointConfig'

ENDPOINT_NAME = 'MultiModelEndpoint'
ENDPOINT_INSTANCE_TYPE = 'ml.m4.xlarge'


In [ ]:
# Identify new model name -pmh
#multi_variant_model_name = 'MultiVariantModel'

mme_model_name = "mme-model-v1"

In [ ]:
# create primary container -pmh
primary_container <- vector(mode="list", length=3)
names(primary_container) <- c('Image', 'ModelDataUrl','Mode')      

primary_container[[1]] <- container
primary_container[[2]] <- model_data
primary_container[[3]] <- 'MultiModel'


In [ ]:
primary_container


In [ ]:
# This is references the AWS managed XGBoost container
#XGBOOST_IMAGE = sagemaker$amazon$amazon_estimator$get_image_uri(session$boto_region_name, 'xgboost', repo_version='latest')

# confirm value of container
primary_container

In [ ]:
# Create new model for multi-variants -pmh
#mme_model <- sm_client$create_model(
#    ModelName = mme_model_name,
#    ExecutionRoleArn = role_arn,
#    PrimaryContainer = primary_container)

#print(paste('MME model ARN: ', mme_model[['ModelArn']]))
#print(mme_model)

In [ ]:
# Create new endpoint configuration for MultiModel -pmh
prod_variant_vector <- vector(mode="list", length=5)
names(prod_variant_vector) <- c('VariantName', 'ModelName', 'InitialInstanceCount', 'InstanceType', 'InitialVariantWeight')

prod_variant_vector[[1]] <- 'AllTraffic'
prod_variant_vector[[2]] <- mme_model_name
prod_variant_vector[[3]] <- 1L
prod_variant_vector[[4]] <- 'ml.t2.medium'
prod_variant_vector[[5]] <- 1L

prod_variant_vector

prod_variant_list <- vector(mode="list", length=1)
prod_variant_list[[1]] = prod_variant_vector

In [ ]:
# Create new endpoint config that supports MultiModel deployments -pmh
response = sm_client$create_endpoint_config(
    EndpointConfigName=ENDPOINT_CONFIG_NAME,
    ProductionVariants=prod_variant_list)

response

In [ ]:
# Create multi-variant Endpoint -pmh
multi_model_endpoint = sm_client$create_endpoint(
    EndpointName=ENDPOINT_NAME,
    EndpointConfigName=ENDPOINT_CONFIG_NAME
    )


In [ ]:
multi_model_endpoint

In [ ]:
# Describe new endpoint, viewing the $EndpointStatus element...
# Re-run this cell until it's status reads -> "InService"  -pmh
endpoint_desc = sm_client$describe_endpoint(
    EndpointName=ENDPOINT_NAME
)

endpoint_desc

### Attempt to create Multi-Model endpoint

In [ ]:
# This is where our MME will read models from on S3.
DEFAULT_BUCKET <- bucket
DATA_PREFIX <- '/multi_model_artifacts/models'

model_data_prefix = paste('s3://', DEFAULT_BUCKET, DATA_PREFIX, sep='')
model_data_prefix

In [ ]:
#mme_model <- estimator$create_model(role=role_arn, image=container)

In [ ]:
# Dump MME Model
print(mme_model_name)
print(model_data_prefix)

In [ ]:
#print(multi_model_endpoint)

In [ ]:
multi_data_model <- sagemaker$multidatamodel$MultiDataModel(
                        name=mme_model_name,
                        model_data_prefix=model_data_prefix,
                        model=multi_model_from_estimator, # passing our model - passes container image needed for the endpoint
                        sagemaker_session=session)
multi_data_model

In [ ]:
predictor = multi_data_model$deploy(initial_instance_count=1,
                       instance_type=ENDPOINT_INSTANCE_TYPE,
                       endpoint_name=ENDPOINT_NAME)

In [ ]:
# List active models - should be none
list(multi_data_model$list_models())

In [ ]:
estimator

In [ ]:
# load model from estimator
#artifact_path = estimator$latest_training_job$describe()['ModelArtifacts']['S3ModelArtifacts']

artifact_path = model_data
print(paste('Model Artifact Loc:', artifact_path))

In [ ]:
#model_name = artifact_path.split('/')[-4]+'.tar.gz'
model_name = 'mme-model.tar.gz'

In [ ]:
# This is copying over the model artifact to the S3 location for the MME.
multi_data_model$add_model(model_data_source=artifact_path, model_data_path=model_name)

In [ ]:
print(multi_data_model$list_models())


In [ ]:
print(estimator$model_data)

### Run test of multi-variant prediction

In [ ]:
# Add content-type and serializer -pmh
multi_model_endpoint$content_type <- 'text/csv'
multi_model_endpoint$serializer <- sagemaker$predictor$csv_serializer

multi_model_endpoint

In [ ]:
abalone_test <- abalone_test[-1]
num_predict_rows <- 500
test_sample <- as.matrix(abalone_test[1:num_predict_rows, ])
dimnames(test_sample)[[2]] <- NULL

In [ ]:
# dump test sample
test_sample

In [ ]:
single_sample <- abalone_test[1,]
single_sample

In [ ]:
csv_sample = '1,0,0,0.44,0.345,0.17,0.4085,0.15,0.0825,0.1515'
csv_sample

In [ ]:
# SageMaker runtime
runtime_sagemaker = paws::sagemakerruntime();


In [ ]:
runtime_response <- runtime_sagemaker$invoke_endpoint(
    EndpointName = multi_model_endpoint,
    ContentType = 'text/csv',
    # TargetModel=mme_model,
    Body = csv_sample,
    )

runtime_response

<h3>Deleting the endpoint</h3>

When you're done with the model, delete the endpoint to avoid incurring deployment costs:

In [ ]:
# session$delete_endpoint(model_endpoint$endpoint)